In [1]:
import mwclient
import time

site = mwclient.Site("en.wikipedia.org")
page = site.pages['Ripple']

In [2]:
revs = list(page.revisions())

In [3]:
revs[0]

OrderedDict([('revid', 1187735781),
             ('parentid', 1184109175),
             ('user', 'Sdf'),
             ('timestamp',
              time.struct_time(tm_year=2023, tm_mon=12, tm_mday=1, tm_hour=2, tm_min=46, tm_sec=16, tm_wday=4, tm_yday=335, tm_isdst=-1)),
             ('comment', '/* Arts and entertainment */')])

In [4]:
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [5]:
revs[0]

OrderedDict([('revid', 38507906),
             ('parentid', 0),
             ('user', 'SRipple'),
             ('timestamp',
              time.struct_time(tm_year=2006, tm_mon=2, tm_mday=6, tm_hour=20, tm_min=43, tm_sec=17, tm_wday=0, tm_yday=37, tm_isdst=-1)),
             ('comment', '')])

In [6]:
from transformers import pipeline
sentament_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentament_pipeline([text[: 250]]) [0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

c:\Users\aivi1\ptu20_ml_reinforcement\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [8]:
edits = {}

for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["'timestamp'"])

    if date not in edits:
        edits[date] = dict(sentiments = list(), edit_count = 0)
        
    edits[date]["edit_count"] += 1

    comment = rev('comment')
    edits[date]["sentiments"].append(find_sentiment(comment))

KeyError: "'timestamp'"

In [ ]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0

    del edits[key]["sentiments"]